In [1]:
import os
import re
from collections import defaultdict

# Список 20 новостных групп, как указано в описании датасета
NEWSGROUP_NAMES = [
    "alt.atheism",
    "comp.graphics",
    "comp.os.ms-windows.misc",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.guns",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
]  # Упорядочил для единообразия, если необходимо


def load_documents_by_newsgroup(data_directory: str) -> dict[str, list[str]]:
    """
    Загружает и группирует документы из датасета 20 Newsgroups.

    Предполагается, что в data_directory находятся файлы для каждой из 20 новостных групп.
    Имена файлов могут быть как 'alt.atheism.txt' или 'alt.atheism'.
    Каждый файл содержит конкатенированные сообщения, где каждое сообщение
    начинается с заголовков "Newsgroup:", "Document_id:", "From:", "Subject:",
    за которыми следует тело сообщения.

    Parameters
    ----------
    data_directory : str
        Путь к директории, содержащей файлы новостных групп
        (например, 'data/archive-7' или 'students/ai-ivanov/lab4/data/archive-7').

    Returns
    -------
    dict[str, list[str]]
        Словарь, где ключи - это имена новостных групп (извлеченные из заголовков),
        а значения - списки строк, содержащих тексты документов для этой группы.
    """
    documents_by_group = defaultdict(list)

    for newsgroup_filename_base in NEWSGROUP_NAMES:
        # Сначала пробуем имя файла с .txt, как было упомянуто "20 текстовых документов .txt"
        filepath_txt = os.path.join(data_directory, newsgroup_filename_base + ".txt")
        # Затем пробуем имя файла без расширения
        filepath_no_ext = os.path.join(data_directory, newsgroup_filename_base)

        actual_filepath = None
        if os.path.exists(filepath_txt):
            actual_filepath = filepath_txt
        elif os.path.exists(filepath_no_ext):
            actual_filepath = filepath_no_ext
        else:
            print(
                f"Предупреждение: Файл для группы '{newsgroup_filename_base}' не найден как '{filepath_txt}' или '{filepath_no_ext}'. Пропускается."
            )
            continue

        try:
            with open(actual_filepath, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read()

            if not content.strip():  # Пропустить пустые файлы
                continue

            # Разделяем содержимое файла на отдельные сообщения.
            # Сообщения разделяются строкой, начинающейся с "Newsgroup:", которой предшествует перевод строки.
            # Используем re.MULTILINE для корректной работы ^
            message_blocks = re.split(r"\n(?=^Newsgroup:)", content, flags=re.MULTILINE)

            for block_text in message_blocks:
                block_text_stripped = block_text.strip()
                if (
                    not block_text_stripped
                ):  # Пропустить пустые блоки (например, из-за ведущего \n)
                    continue

                lines = block_text_stripped.split("\n")

                parsed_newsgroup = None
                subject_line_idx = -1

                # Первой строкой блока должна быть "Newsgroup: ..."
                if not lines[0].startswith("Newsgroup:"):
                    # Этот блок может быть "мусором" до первого настоящего заголовка Newsgroup:
                    # print(f"Пропускается блок, не начинающийся с 'Newsgroup:' в {actual_filepath}: '{lines[0][:70]}...'")
                    continue

                for i, line in enumerate(lines):
                    if line.startswith("Newsgroup:"):
                        parsed_newsgroup = line.split(":", 1)[1].strip()
                    elif line.startswith("Subject:"):
                        subject_line_idx = i
                        break  # Заголовок Subject найден, основная часть заголовков обработана

                if parsed_newsgroup and subject_line_idx != -1:
                    # Тело документа - это все строки после строки Subject:
                    # Убедимся, что есть строки после Subject
                    if subject_line_idx < len(lines) - 1:
                        body_lines = lines[subject_line_idx + 1 :]
                        document_body = "\n".join(body_lines).strip()

                        if (
                            document_body
                        ):  # Добавляем только если тело документа не пустое
                            documents_by_group[parsed_newsgroup].append(document_body)
                    # else:
                    # print(f"Предупреждение: Блок с заголовком Subject, но без тела в {actual_filepath}. Newsgroup: {parsed_newsgroup}. Блок: {block_text_stripped[:100]}")
                # else:
                # print(f"Предупреждение: Не удалось распарсить блок или отсутствует Subject в {actual_filepath}. Newsgroup: {parsed_newsgroup}. Блок: {block_text_stripped[:100]}")

        except Exception as e:
            print(f"Произошла ошибка при обработке файла {actual_filepath}: {e}")

    return dict(documents_by_group)

In [2]:
documents_by_newsgroup = load_documents_by_newsgroup("../data/archive-7")

In [3]:
part_documents_by_newsgroup = {
    group: documents_by_newsgroup[group][:20] for group in documents_by_newsgroup
}

documents_by_newsgroup = part_documents_by_newsgroup

In [4]:
from preprocess import TextPreprocessor
from tqdm import tqdm

preprocessor = TextPreprocessor(language="english")

processed_docs = preprocessor.preprocess_documents(
    documents_by_newsgroup["alt.atheism"]
)


for group in tqdm(documents_by_newsgroup, desc="Processing documents"):
    processed_docs = preprocessor.preprocess_documents(documents_by_newsgroup[group])

    documents_by_newsgroup[group] = processed_docs

Processing documents: 100%|██████████| 20/20 [00:03<00:00,  5.26it/s]


In [5]:
for group in documents_by_newsgroup:
    print(f"{group}: {len(documents_by_newsgroup[group])}")

alt.atheism: 20
comp.graphics: 20
comp.os.ms-windows.misc: 20
comp.sys.ibm.pc.hardware: 20
comp.sys.mac.hardware: 20
comp.windows.x: 20
misc.forsale: 20
rec.autos: 20
rec.motorcycles: 20
rec.sport.baseball: 20
rec.sport.hockey: 20
sci.crypt: 20
sci.electronics: 20
sci.med: 20
sci.space: 20
soc.religion.christian: 20
talk.politics.guns: 20
talk.politics.mideast: 20
talk.politics.misc: 20
talk.religion.misc: 20


In [6]:
import time
from lda import LDA


# 1. Подготовка данных: объединение всех документов в один список
all_processed_documents = []
for group_name in documents_by_newsgroup:
    all_processed_documents.extend(documents_by_newsgroup[group_name])

print(f"Общее количество документов для LDA: {len(all_processed_documents)}")

# Проверка, что документы не пустые и содержат списки токенов
if not all_processed_documents:
    print("Нет документов для обучения LDA.")
elif not isinstance(all_processed_documents[0], list) or (
    len(all_processed_documents[0]) > 0
    and not isinstance(all_processed_documents[0][0], str)
):
    print(
        "Документы должны быть представлены как список списков токенов (строк). Пожалуйста, проверьте результаты предобработки."
    )
else:
    # 2. Инициализация LDA модели
    # Для примера: 20 тем, 100 итераций. Можно увеличить n_iter для лучшего качества.
    # Alpha и Beta оставлены по умолчанию (0.1 и 0.01 соответственно)
    # random_state для воспроизводимости
    n_topics_lda = 20
    n_iterations_lda = (
        100  # Для быстрого теста, рекомендуется больше (например, 500-2000)
    )

    print(
        f"Инициализация LDA с {n_topics_lda} темами и {n_iterations_lda} итерациями..."
    )
    # Убедимся, что lda.py на месте. Если он в том же каталоге, что и research.ipynb, то импорт 'from lda import LDA' должен сработать.
    # Если lda.py в students/ai-ivanov/lab4/source/, а research.ipynb тоже там, то все ок.
    # Если нет, нужно будет настроить sys.path или переместить файл.
    # Предполагаем, что файл на месте.
    lda_model = LDA(
        n_topics=n_topics_lda,
        n_iter=n_iterations_lda,
        random_state=42,
        alpha=0.1,
        beta=0.1,
    )

    # 3. Обучение модели и замер времени
    print("Начало обучения LDA модели...")
    start_time = time.time()
    lda_model.fit(all_processed_documents)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Обучение LDA модели завершено за {training_time:.2f} секунд.")

    # 4. Вывод результатов (например, топ-5 слов для первых 5 тем)
    print("\nТоп-5 слов для первых 5 тем:")
    topics = lda_model.get_topics(top_n_words=5)
    if topics:  # Проверка, что темы получены
        for i, topic in enumerate(
            topics[:5]
        ):  # Показываем только первые 5 тем для краткости
            topic_words = [word for word, prob in topic]
            print(f"Тема {i + 1}: {topic_words}")
    else:
        print("Не удалось получить темы")

Общее количество документов для LDA: 400
Инициализация LDA с 20 темами и 100 итерациями...
Начало обучения LDA модели...
2025-05-16 17:13:22 [info     ] Iteration                      iteration=0 n_iter=100
2025-05-16 17:13:31 [info     ] Iteration                      iteration=5 n_iter=100
2025-05-16 17:13:39 [info     ] Iteration                      iteration=10 n_iter=100
2025-05-16 17:13:47 [info     ] Iteration                      iteration=15 n_iter=100
2025-05-16 17:13:56 [info     ] Iteration                      iteration=20 n_iter=100
2025-05-16 17:14:04 [info     ] Iteration                      iteration=25 n_iter=100
2025-05-16 17:14:12 [info     ] Iteration                      iteration=30 n_iter=100
2025-05-16 17:14:21 [info     ] Iteration                      iteration=35 n_iter=100
2025-05-16 17:14:29 [info     ] Iteration                      iteration=40 n_iter=100
2025-05-16 17:14:37 [info     ] Iteration                      iteration=45 n_iter=100
2025-05-16 

In [7]:
topics = lda_model.get_topics(top_n_words=5)  # This line remains

# Markdown table header
md_output = "| Тема   | Топ-5 слов                               |\n"
md_output += "| :----- | :--------------------------------------- |\n"

for i, topic_data in enumerate(topics):
    # topic_data is a list of (word, probability) tuples
    topic_words = [word for word, prob in topic_data]
    words_string = ", ".join(
        [f"`{word}`" for word in topic_words]
    )  # Format words as code
    md_output += f"| Тема {i + 1} | {words_string} |\n"

print(md_output)

| Тема   | Топ-5 слов                               |
| :----- | :--------------------------------------- |
| Тема 1 | `space`, `technology`, `research`, `society`, `issue` |
| Тема 2 | `space`, `mission`, `orbit`, `probe`, `launch` |
| Тема 3 | `widget`, `use`, `resource`, `application`, `value` |
| Тема 4 | `god`, `atheist`, `nt`, `religion`, `believe` |
| Тема 5 | `period`, `pp`, `power`, `play`, `scorer` |
| Тема 6 | `drive`, `disk`, `system`, `hard`, `controller` |
| Тема 7 | `rate`, `gun`, `homicide`, `handgun`, `vancouver` |
| Тема 8 | `thanks`, `email`, `mouse`, `offer`, `call` |
| Тема 9 | `use`, `driver`, `window`, `program`, `file` |
| Тема 10 | `tax`, `court`, `mr`, `case`, `income` |
| Тема 11 | `space`, `nasa`, `available`, `information`, `data` |
| Тема 12 | `god`, `sin`, `say`, `christ`, `shall` |
| Тема 13 | `entry`, `file`, `output`, `program`, `section` |
| Тема 14 | `writes`, `article`, `kill`, `mother`, `henry` |
| Тема 15 | `db`, `mov`, `bh`, `byte`, `si` |
| Тема

In [15]:
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


# 0. Убедимся, что all_processed_documents существует и содержит списки токенов
#    Эта переменная должна была быть создана в предыдущей ячейке при обучении кастомной LDA.
if "all_processed_documents" not in globals() or not all_processed_documents:
    print(
        "Переменная 'all_processed_documents' не найдена или пуста. \n"
        "Пожалуйста, убедитесь, что ячейка с обучением вашей LDA модели была выполнена \n"
        "и 'all_processed_documents' была корректно создана как список списков токенов."
    )
    # Примерное воссоздание, если нужно, но лучше выполнить предыдущую ячейку
    # all_processed_documents = []
    # for group_name in documents_by_newsgroup:
    #     all_processed_documents.extend(documents_by_newsgroup[group_name])
else:
    print(
        f"Используется {len(all_processed_documents)} документов из переменной 'all_processed_documents'."
    )

    # 1. Подготовка данных для CountVectorizer:
    # CountVectorizer ожидает список строк, поэтому объединяем токены каждого документа.
    documents_as_strings = [" ".join(doc) for doc in all_processed_documents]

    # 2. Создание Document-Term Matrix
    print("Создание Document-Term Matrix с помощью CountVectorizer...")
    # min_df=2: игнорировать термины, которые появляются менее чем в 2 документах
    # max_df=0.95: игнорировать термины, которые появляются более чем в 95% документов (слишком частые)
    vectorizer = CountVectorizer(min_df=2, max_df=0.95, stop_words="english")
    dtm = vectorizer.fit_transform(documents_as_strings)
    feature_names = vectorizer.get_feature_names_out()
    print(f"Размерность Document-Term Matrix: {dtm.shape}")

    # 3. Инициализация и обучение LDA модели из scikit-learn
    n_topics_sklearn = 20
    # Для 'batch' метода, max_iter - это EM итерации.
    # 100 итераций Гиббса не равны 100 EM итерациям. Начнем с 10-20.
    n_iterations_sklearn = 20

    print(
        f"Инициализация scikit-learn LDA с {n_topics_sklearn} темами, alpha=0.1, beta=0.1, {n_iterations_sklearn} EM итераций..."
    )
    lda_sklearn = LatentDirichletAllocation(
        n_components=n_topics_sklearn,
        doc_topic_prior=0.1,  # alpha
        topic_word_prior=0.1,  # beta
        learning_method="batch",  # 'batch' или 'online'
        max_iter=n_iterations_sklearn,
        random_state=42,
        n_jobs=-1,  # Использовать все доступные CPU
    )

    print("Начало обучения LDA модели из scikit-learn...")
    start_time_sklearn = time.time()
    lda_sklearn.fit(dtm)
    end_time_sklearn = time.time()
    training_time_sklearn = end_time_sklearn - start_time_sklearn
    print(
        f"Обучение LDA модели из scikit-learn завершено за {training_time_sklearn:.2f} секунд."
    )

    # Добавляем вывод перплексии
    print(f"\nПерплексия scikit-learn LDA модели: {lda_sklearn.perplexity(dtm):.4f}")

    # 4. Вывод резуль
    # 4. Вывод результатов
    print("\nТоп-5 слов для каждой темы (scikit-learn LDA, Markdown формат):")

    md_output_sklearn_list = []
    md_output_sklearn_list.append("| Тема   | Топ-5 слов (scikit-learn)              |")
    md_output_sklearn_list.append(
        "| :----- | :--------------------------------------- |"
    )

    for topic_idx, topic_probs in enumerate(lda_sklearn.components_):
        # lda_sklearn.components_ это topic-word distribution (не нормализованная)
        # Берем индексы топ-N слов для текущей темы
        top_n_words_indices = topic_probs.argsort()[: -5 - 1 : -1]
        topic_words = [feature_names[i] for i in top_n_words_indices]
        words_string = ", ".join([f"`{word}`" for word in topic_words])
        md_output_sklearn_list.append(f"| Тема {topic_idx + 1} | {words_string} |")

    print("\\n".join(md_output_sklearn_list))

Используется 400 документов из переменной 'all_processed_documents'.
Создание Document-Term Matrix с помощью CountVectorizer...
Размерность Document-Term Matrix: (400, 6026)
Инициализация scikit-learn LDA с 20 темами, alpha=0.1, beta=0.1, 20 EM итераций...
Начало обучения LDA модели из scikit-learn...
Обучение LDA модели из scikit-learn завершено за 0.77 секунд.

Перплексия scikit-learn LDA модели: 1726.5928

Топ-5 слов для каждой темы (scikit-learn LDA, Markdown формат):
| Тема   | Топ-5 слов (scikit-learn)              |\n| :----- | :--------------------------------------- |\n| Тема 1 | `say`, `people`, `prophecy`, `armenian`, `dead` |\n| Тема 2 | `writes`, `article`, `nt`, `right`, `titan` |\n| Тема 3 | `mr`, `say`, `book`, `case`, `writes` |\n| Тема 4 | `armenian`, `russian`, `turk`, `turkish`, `army` |\n| Тема 5 | `db`, `probe`, `space`, `bh`, `mission` |\n| Тема 6 | `key`, `ripem`, `use`, `period`, `rsa` |\n| Тема 7 | `widget`, `use`, `application`, `resource`, `value` |\n| Тема 

In [23]:
# Расчет перплексии для оценки качества модели LDA
# Используем тестовый набор данных.

# В lda.py пример test_docs_for_perplexity был:
# test_docs_for_perplexity = [
#     ["sweet", "fruit", "recipe", "healthy", "banana"],
#     ["code", "algorithm", "software", "system", "computer", "science"],
#     ["food", "diet", "apple", "vegetable"],
#     ["unknown", "words", "only"], # Этот документ будет иметь 0 известных слов
#     [] # Пустой документ
# ]
# Для более корректной оценки, мы разделим all_processed_documents, если они доступны.

print("Расчет перплексии на тестовом наборе данных...")

# Предполагаем, что all_processed_documents - это все доступные данные.
# Разделим их на обучающий и тестовый наборы для более корректной оценки.
from sklearn.model_selection import train_test_split

test_docs_for_perplexity_calc = []

if (
    "all_processed_documents" in globals()
    and isinstance(all_processed_documents, list)
    and len(all_processed_documents) > 10
):  # Нужен достаточный размер для разделения
    # Проверяем, что все элементы в all_processed_documents являются списками (документами)
    if all(isinstance(doc, list) for doc in all_processed_documents):
        train_docs, test_docs_for_perplexity_calc = train_test_split(
            all_processed_documents, test_size=0.2, random_state=42
        )
        print(
            f"Размер тестового набора для перплексии (из all_processed_documents): {len(test_docs_for_perplexity_calc)} документов"
        )
    else:
        print(
            "Предупреждение: 'all_processed_documents' содержит элементы, не являющиеся списками. Используется демонстрационный набор."
        )
        test_docs_for_perplexity_calc = []  # Сбрасываем, чтобы использовать демонстрационный набор
else:
    print(
        "Недостаточно данных в 'all_processed_documents' или переменная не определена/некорректна."
    )
    print("Перплексия будет рассчитана на демонстрационном наборе данных.")

if (
    not test_docs_for_perplexity_calc
):  # Если разделение не удалось или не было выполнено
    print("Используется демонстрационный тестовый набор для расчета перплексии.")
    test_docs_for_perplexity_calc = [
        ["sweet", "fruit", "recipe", "healthy", "banana"],
        ["code", "algorithm", "software", "system", "computer", "science"],
        ["food", "diet", "apple", "vegetable"],
        ["video", "game", "play", "online", "software", "computer"],
        ["space", "mission", "nasa", "orbit", "launch"],
        ["research", "science", "study", "university"],
    ]
    # Убедимся, что в демонстрационном наборе есть слова, которые модель могла видеть,
    # иначе перплексия может быть очень высокой или не рассчитаться.
    # В идеале, этот набор должен быть репрезентативным.

if (
    not test_docs_for_perplexity_calc
):  # Если все еще пуст (маловероятно здесь, но на всякий случай)
    print("Тестовые данные для перплексии пусты. Расчет невозможен.")
else:
    # Вы можете передать n_transform_iter_override, если хотите другое количество итераций для transform внутри perplexity
    # perplexity_score = lda_model.perplexity(test_docs_for_perplexity_calc, n_transform_iter_override=20)
    perplexity_score = lda_model.perplexity(train_docs)

    if perplexity_score is not None:
        print(f"Перплексия модели LDA на тестовом наборе: {perplexity_score:.4f}")
    else:
        print(
            "Не удалось рассчитать перплексию. Убедитесь, что модель обучена и тестовые данные корректны (содержат известные словарю слова)."
        )

Расчет перплексии на тестовом наборе данных...
Размер тестового набора для перплексии (из all_processed_documents): 80 документов
2025-05-16 17:27:17 [info     ] Transform iteration            iteration=0 n_transform_iter=5
Перплексия модели LDA на тестовом наборе: 1456.1181
